In [ ]:
# Uncomment to run in google colab
#!sudo apt-get install libcairo2-dev
#!pip install python-igraph --quiet
#!pip install pycairo --quiet

# Session 7 - iGraph

This notebook is for using the python bindings for iGraph, if you prefer R (ugh!) or even C/C++ (OMG!) refer to the corresponding documentation.

If you want to install python-igraph on windows or mac you should follow the documentation instructions http://igraph.org/python/#startpy

To generate scalable vector graphs (SVG) with the plot methods you need *pycairo* (it is not instalable using pip, you have to download it from http://cairographics.org/pycairo/) and the library *cairo* has to be installed in your linux). We will avoid using pycairo generating png files instead.

In this notebook it is also used numpy and matplotlib, they also can be installed using pip

Apart from this notebook, the tutorial for iGraph is a good starting point http://igraph.org/python/doc/tutorial/tutorial.html

In [3]:
from igraph import Graph

Creating a graph is simple, this is an **empty** graph (print returns a representation of the graph)

In [ ]:
g = Graph()
print(g)

We can add vertices like for example, vertices from [0..3]

In [ ]:
g.add_vertices(4)
print(g)

and some edges among them (as a list of pairs)

In [ ]:
g.add_edges([(1,2), (1,3), (2,3), (3, 0)])
print(g)

Class Graph provides a couple of iterators to traverse the vertices (**vs**) and the edges (**es**)

In [ ]:
for v in g.vs:
    print(v)

In [ ]:
for e in g.es:
    print(e)

Properties can be attached to the vertices (and the edges)

In [ ]:
for i, v in enumerate(g.vs):
    v['name'] = str(i)
for v in g.vs:
    print(v.attributes())

In [ ]:
import random
for edge in g.es:
    edge['weight'] = random.randint(1,10)
for edge in g.es:
    print(edge['weight'])

Class Graph has methods to compute properties of a graph and many useful algorithms (http://igraph.org/python/doc/igraph.Graph-class.html)

In [ ]:
g.degree()

In [ ]:
g.edge_betweenness()

Plotting the graph is a little bit more complicated because by default it uses the Cairo library (that it is not installed in the lab PCs), we can circunvent this problem by generatin a png file and loading it in the notebook. There are different styles and layouts for plotting a graph, you can look for the parameters in the documentation of the plot function in the Graph class.

In [9]:
from IPython import display
from igraph import plot

g.vs['label'] = g.vs['name']
g.es['width'] = g.es['weight']
plot(g, layout = g.layout_circle(),target="./igraph_demo.png")
display.Image(filename="./igraph_demo.png")

NameError: name 'g' is not defined

## Graph Generators

iGraph implements several graph generators (Erdos-Renyi, Barabasi, Watts-Strogratz, ...), different layouts have been used for plotting the graphs (you can look up in the documentation the different alternatives and parameters)

In [ ]:
erdos = Graph.Erdos_Renyi(100,0.05)
r = plot(erdos, layout = erdos.layout_kamada_kawai(),target="./erdos.png")
display.Image(filename="./erdos.png")

In [ ]:
barabasi = Graph.Barabasi(100,1)
r = plot(barabasi, layout = barabasi.layout_fruchterman_reingold(),target="./barabasi.png")
display.Image(filename="./barabasi.png")

In [4]:
watts = Graph.Watts_Strogatz(1,100,2,0.05)
r = plot(watts, layout = watts.layout_lgl(),target="./watts.png")
display.Image(filename="watts.png")

NameError: name 'plot' is not defined

## Measuring graphs

The following cells replicates more or less what you have in the session documentation for the lattice graph

In [ ]:
lattice = Graph.Lattice([10,10])
r = plot(lattice, layout=lattice.layout_kamada_kawai(),target="./lattice.png")
display.Image(filename="./lattice.png")

In [ ]:
lattice.average_path_length()

In [ ]:
lattice.diameter()

In [ ]:
lattice.rewire_edges(0.2)

In [ ]:
lattice.average_path_length()

In [ ]:
lattice.diameter()

The following is for the watts, erdos and ring graphs and the different measures over graphs

In [ ]:
watts.transitivity_undirected()

In [ ]:
p_hat = len(watts.es)/(len(watts.vs)*len(watts.vs)/2)
p_hat

In [ ]:
erdos.transitivity_undirected()

In [ ]:
ring = Graph.Ring(10)
r = plot(ring, layout=ring.layout_kamada_kawai(),target="./ring.png")
display.Image(filename="./ring.png")

In [ ]:
ring.degree()

In [ ]:
barabasi = Graph.Barabasi(1000,3)
p_hat = len(barabasi.es)/((len(barabasi.vs)-1)*len(barabasi.vs)/2.0)
p_hat

In [ ]:
erdos = Graph.Erdos_Renyi(1000,p_hat)
p_hat

In [ ]:
from numpy import histogram, max
h, b = histogram(erdos.degree(), bins=max(erdos.degree()), density=True)
print(h)
print(b)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
r = plt.hist(erdos.degree(), bins=max(erdos.degree()))

In [ ]:
r = plt.plot(b[1:], h, 'o' )

In [ ]:
r = plt.hist(barabasi.degree(), bins=max(barabasi.degree()))

In [ ]:
h, b = histogram(barabasi.degree(), bins=max(barabasi.degree()), density=True)
r = plt.plot(b[1:], h, 'o' )

## Communities

This follows the part about social graphs measures and comunity detection.

First we creeate a graph following the Erdos-Reny model

In [ ]:
from igraph import plot
erdos = Graph.Erdos_Renyi(20,0.2)

r = plot(erdos, layout = erdos.layout_kamada_kawai(),target="./erdos.png")
display.Image(filename="./erdos.png")

We can measure vertices **betweenness**

In [ ]:
erdos.betweenness()

Vertices **degree**

In [ ]:
erdos.degree()

Vertices **closeness**

In [ ]:
erdos.closeness()

nodes **pagerank**

In [ ]:
erdos.pagerank()

iGraph also has several community detection algorithms, you can look them up in the Graph class documentation to see how they work and how to use them.

The algorithm return different objects (clusters or dendrograms) that have different methods for obtaining what you need.

The method **community_edge_betweenness()** returns a dendrogram object that allows to extract the clusters. You can also plot dendrograms and clusters.

In [ ]:
com = erdos.community_edge_betweenness()
r = plot(com,target="./erdos-dendro.png", orientation='bottom-top')
print ('Clusters:', com.optimal_count) # Optimal number of clusters from the dendrogram
display.Image(filename="./erdos-dendro.png")

the *as_clustering* method of the dendrogram object returns a graph with the clusters

In [ ]:
r = plot(com.as_clustering(), layout = erdos.layout_kamada_kawai(),target="./com-erdos.png",mark_groups=True)
display.Image(filename="./com-erdos.png")

You can also obtain the clusters and the vertices in the clusters

In [ ]:
for d in com.as_clustering():
    print(d)

## Social experiment

You have with the session documentation the file 'caimlab.txt'. This is the graph of the laboratory 'partnerships' of the previous lab sessions.

The graph has been anonymized.

In [ ]:
# Uncomment to run in google colab
#!wget https://www.cs.upc.edu/~caim/lab/session7networks.zip
#!unzip session7networks.zip caimlab*.txt

In [ ]:
g = Graph(directed=False)
g = g.Load('./caimlab.txt', format='edgelist', directed=False)
r = plot(g, layout = g.layout_kamada_kawai(),target="./caimlab-graph.png")
display.Image(filename="./caimlab-graph.png")

We can compute the communities in this social graph using the edge betweenness

In [ ]:
#clust = g.community_optimal_modularity()
#clust = g.community_label_propagation()
#clust = g.community_leading_eigenvector()
#clust = g.community_infomap()
clust = g.community_multilevel()

In [ ]:
print(clust)

In [ ]:
r = plot(clust,target="./caimlab-clust.png",mark_groups=True)
display.Image(filename="./caimlab-clust.png")

In [ ]:
clust = g.community_edge_betweenness()

In [ ]:
print(clust)

In [ ]:
r = plot(clust, layout = g.layout_lgl(),target="./caimlab-eb-den.png")
display.Image(filename="./caimlab-eb-den.png")

In [ ]:
clust = clust.as_clustering()

r = plot(clust,target="./caimlab-clust.png",mark_groups=True)
display.Image(filename="./caimlab-clust.png")